# Entrenamiento con Simple Transformers del model Distill BERT

## Paso 1: Cargar datos en Google Colab

## Paso 2: Instalar librerías

In [1]:
!pip install transformers evaluate torch --quiet
!pip install simpletransformers transformers datasets huggingface_hub scikit-learn
!pip install evaluate --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.3/43.3 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 3.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 330.8/330.8 kB 30.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 71.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 152.0 MB/s eta 0:00:00
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16162 sha256=bbce20e1444d420fcda437152bb559df55ebc3b050b023da7528f5bc15b3c88d
  Stored in directory: /root/.cache/pip/wheels/5f/b8/73/0b2c1a76b701a677653dd79ece07cfabd7457989dbfbdcd8d7
Successfully built seqeval


## Paso 3: Cargar librerías

In [2]:
from transformers import pipeline, AutoTokenizer, AutoModelForQuestionAnswering
import evaluate
import json
import os
import shutil
import requests


from tqdm import tqdm
from simpletransformers.question_answering import QuestionAnsweringModel, QuestionAnsweringArgs
from sklearn.model_selection import train_test_split
from google.colab import files

In [3]:
import logging
logging.getLogger("transformers.modeling_utils").setLevel(logging.ERROR)

In [4]:
import torch
print("GPU available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("GPU name:", torch.cuda.get_device_name(0))

GPU available: True
GPU name: Tesla T4


## Paso 4: Cargar datos

In [5]:
# URLs of the files
urls = {
    "eval_colombia_mexico_dataset.json": "https://github.com/BlueAutomata/tesis-optimizacion-de-modelos-de-question-answering/raw/refs/heads/master/src/datasets/exploration_datasets/gold/eval_colombia_mexico_dataset.json",
    "train_colombia_mexico_dataset.json": "https://github.com/BlueAutomata/tesis-optimizacion-de-modelos-de-question-answering/raw/refs/heads/master/src/datasets/exploration_datasets/gold/train_colombia_mexico_dataset.json"
}

# Dictionary to store the loaded JSON data
datasets = {}

for filename, url in urls.items():
    # Download the file
    response = requests.get(url)
    if response.status_code == 200:
        # Save locally
        with open(filename, "wb") as f:
            f.write(response.content)
        # Load JSON into Python
        datasets[filename] = response.json()
        print(f"{filename} downloaded and loaded successfully!")
    else:
        print(f"Failed to download {filename}. Status code: {response.status_code}")

eval_colombia_mexico_dataset.json downloaded and loaded successfully!
train_colombia_mexico_dataset.json downloaded and loaded successfully!


In [6]:
with open("train_colombia_mexico_dataset.json", "r", encoding="utf-8") as f:
    train_dataset = json.load(f)

print(f"✅ Loaded {len(train_dataset)} records successfully!")

✅ Loaded 1 records successfully!


In [7]:
with open("eval_colombia_mexico_dataset.json", "r", encoding="utf-8") as f:
    eval_dataset = json.load(f)

print(f"✅ Loaded {len(eval_dataset)} records successfully!")

✅ Loaded 1 records successfully!


In [8]:
train_dataset = train_dataset["data"]

In [9]:
eval_dataset = eval_dataset["data"]

In [10]:
# 🔧 1️⃣ Flatten your dataset so each row has 'context' and 'qas'
def flatten_squad(dataset):
    new_data = []
    for article in dataset:
        for para in article["paragraphs"]:
            new_data.append({
                "context": para["context"],
                "qas": para["qas"]
            })
    return new_data

In [11]:
train_data = flatten_squad(train_dataset)

In [12]:
eval_data = flatten_squad(eval_dataset)

In [13]:
print(f"✅ Training samples: {len(train_data)}")
print(f"✅ Eval samples: {len(eval_data)}")

✅ Training samples: 4320
✅ Eval samples: 3846


## Paso 5: Definir hiperparámetros

In [14]:
model_args = QuestionAnsweringArgs()

# ========================
# ⚙️ Training configuration
# ========================
model_args.train_batch_size = 8
model_args.eval_batch_size = 8
model_args.num_train_epochs = 4                # slightly longer training for better convergence
model_args.learning_rate = 3e-5                # standard for BERT fine-tuning
model_args.gradient_accumulation_steps = 4     # effective batch size = 8 × 4 = 32
model_args.warmup_ratio = 0.1
model_args.weight_decay = 0.01                 # regularization to prevent overfitting
model_args.overwrite_output_dir = True

# ========================
# 🧠 Evaluation & Early Stopping
# ========================
model_args.evaluate_during_training = True
model_args.evaluate_during_training_steps = 500
model_args.evaluate_during_training_verbose = True
model_args.save_model_every_epoch = False
model_args.save_eval_checkpoints = True
model_args.save_best_model = True
model_args.metric_for_best_model = "f1"
model_args.greater_is_better = True
model_args.early_stopping_metric = "eval_loss"
model_args.early_stopping_patience = 2
model_args.early_stopping_consider_epochs = True

# ========================
# 📄 Sequence & Context Settings
# ========================
model_args.max_seq_length = 384       # total sequence length
model_args.doc_stride = 128           # sliding window overlap
model_args.max_query_length = 64      # question length
model_args.max_answer_length = 30

# ========================
# 🧩 Output & Logging
# ========================
model_args.output_dir = "./outputs/"
model_args.best_model_dir = "./outputs/best_model/"
model_args.logging_steps = 100
model_args.manual_seed = 42

# ========================
# ⚡ Resource Handling
# ========================
model_args.use_multiprocessing = False             # safer for notebooks
model_args.fp16 = torch.cuda.is_available()        # enable mixed precision if GPU available
model_args.dataloader_num_workers = 2              # small boost in performance
model_args.save_steps = -1                         # don't save intermediate steps

print("✅ Model arguments configured successfully!")

✅ Model arguments configured successfully!


## Paso 6: Cargar el modelo

In [15]:
model_original = QuestionAnsweringModel(
    model_type="bert",
    model_name="mrm8488/distill-bert-base-spanish-wwm-cased-finetuned-spa-squad2-es",  # BETO
    args=model_args,
    use_cuda=torch.cuda.is_available()
)

config.json:   0%|          | 0.00/465 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/439M [00:00<?, ?B/s]

The following layers were not sharded: bert.encoder.layer.*.output.dense.weight, bert.embeddings.token_type_embeddings.weight, bert.encoder.layer.*.attention.self.value.weight, bert.encoder.layer.*.attention.output.LayerNorm.weight, bert.encoder.layer.*.attention.output.LayerNorm.bias, bert.encoder.layer.*.intermediate.dense.weight, bert.encoder.layer.*.output.LayerNorm.weight, bert.encoder.layer.*.attention.output.dense.weight, bert.encoder.layer.*.attention.self.key.bias, qa_outputs.bias, bert.embeddings.position_embeddings.weight, bert.encoder.layer.*.output.dense.bias, bert.encoder.layer.*.output.LayerNorm.bias, bert.encoder.layer.*.attention.output.dense.bias, bert.encoder.layer.*.attention.self.key.weight, bert.embeddings.LayerNorm.weight, bert.encoder.layer.*.attention.self.query.weight, bert.embeddings.LayerNorm.bias, qa_outputs.weight, bert.encoder.layer.*.intermediate.dense.bias, bert.encoder.layer.*.attention.self.query.bias, bert.embeddings.word_embeddings.weight, bert.enco

tokenizer_config.json:   0%|          | 0.00/135 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/439M [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [16]:
model = QuestionAnsweringModel(
    model_type="bert",
    model_name="mrm8488/distill-bert-base-spanish-wwm-cased-finetuned-spa-squad2-es",  # BETO
    args=model_args,
    use_cuda=torch.cuda.is_available()
)

The following layers were not sharded: bert.encoder.layer.*.output.dense.weight, bert.embeddings.token_type_embeddings.weight, bert.encoder.layer.*.attention.self.value.weight, bert.encoder.layer.*.attention.output.LayerNorm.weight, bert.encoder.layer.*.attention.output.LayerNorm.bias, bert.encoder.layer.*.intermediate.dense.weight, bert.encoder.layer.*.output.LayerNorm.weight, bert.encoder.layer.*.attention.output.dense.weight, bert.encoder.layer.*.attention.self.key.bias, qa_outputs.bias, bert.embeddings.position_embeddings.weight, bert.encoder.layer.*.output.dense.bias, bert.encoder.layer.*.output.LayerNorm.bias, bert.encoder.layer.*.attention.output.dense.bias, bert.encoder.layer.*.attention.self.key.weight, bert.embeddings.LayerNorm.weight, bert.encoder.layer.*.attention.self.query.weight, bert.embeddings.LayerNorm.bias, qa_outputs.weight, bert.encoder.layer.*.intermediate.dense.bias, bert.encoder.layer.*.attention.self.query.bias, bert.embeddings.word_embeddings.weight, bert.enco

## Paso 7: Entrenamiento del modelo

In [17]:
model.train_model(train_data, eval_data=eval_data)

add example index and unique id: 100%|██████████| 19487/19487 [00:00<00:00, 670069.45it/s]


Epoch:   0%|          | 0/4 [00:00<?, ?it/s]

/usr/local/lib/python3.12/dist-packages/simpletransformers/question_answering/question_answering_model.py:697: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = amp.GradScaler()


Running Epoch 1 of 4:   0%|          | 0/13019 [00:00<?, ?it/s]

/usr/local/lib/python3.12/dist-packages/simpletransformers/question_answering/question_answering_model.py:720: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast():


convert squad examples to features:   0%|          | 0/8352 [00:00<?, ?it/s]

convert squad examples to features:   0%|          | 1/8352 [00:28<65:44:02, 28.34s/it]

convert squad examples to features:   8%|▊         | 697/8352 [00:30<03:58, 32.16it/s] 

convert squad examples to features:  17%|█▋        | 1393/8352 [00:37<02:12, 52.65it/s]

convert squad examples to features:  33%|███▎      | 2785/8352 [00:38<00:41, 134.48it/s]

convert squad examples to features:  33%|███▎      | 2785/8352 [00:50<00:41, 134.48it/s]

convert squad examples to features:  50%|█████     | 4177/8352 [00:56<00:42, 98.97it/s] 

convert squad examples to features:  58%|█████▊    | 4873/8352 [00:58<00:29, 119.06it/s]

convert squad examples to features: 

...



convert squad examples to features:   0%|          | 0/8352 [00:00<?, ?it/s]

convert squad examples to features:   0%|          | 1/8352 [00:34<80:38:05, 34.76s/it]

convert squad examples to features:  25%|██▌       | 2089/8352 [00:39<01:25, 73.01it/s]

convert squad examples to features:  42%|████▏     | 3481/8352 [00:45<00:44, 109.63it/s]

convert squad examples to features:  42%|████▏     | 3481/8352 [00:55<00:44, 109.63it/s]

convert squad examples to features:  50%|█████     | 4177/8352 [01:00<00:51, 80.76it/s] 

convert squad examples to features:  67%|██████▋   | 5569/8352 [01:04<00:23, 118.32it/s]

convert squad examples to features:  83%|████████▎ | 6961/8352 [01:05<00:07, 181.76it/s]

convert squad examples to features: 100%|██████████| 8352/8352 [01:20<00:00, 103.85it/s]


add example index and unique id: 100%|██████████| 8352/8352 [00:00<00:00, 405792.24it/s]


Running Evaluation:   0%|          | 0/5528 [00:00<?, ?it/s]


convert squad examples to features: 100%|██████████| 8352/8352 [01:28<00:00, 94.34it/s]

add example index and unique id: 100%|██████████| 8352/8352 [00:00<00:00, 366025.40it/s]


Running Evaluation:   0%|          | 0/5528 [00:00<?, ?it/s]

(13016,
 {'global_step': [500,
   1000,
   1500,
   2000,
   2500,
   3000,
   3254,
   3500,
   4000,
   4500,
   5000,
   5500,
   6000,
   6500,
   6508,
   7000,
   7500,
   8000,
   8500,
   9000,
   9500,
   9762,
   10000,
   10500,
   11000,
   11500,
   12000,
   12500,
   13000,
   13016],
  'correct': [5192,
   5463,
   5526,
   5609,
   5640,
   5707,
   5637,
   5561,
   5679,
   5686,
   5694,
   5658,
   5713,
   5764,
   5750,
   5650,
   5709,
   5674,
   5708,
   5697,
   5730,
   5686,
   5703,
   5722,
   5691,
   5717,
   5727,
   5703,
   5725,
   5726],
  'similar': [2712,
   2464,
   2389,
   2209,
   2140,
   2215,
   2331,
   2329,
   2159,
   2172,
   2249,
   2215,
   2228,
   2100,
   2121,
   2199,
   2131,
   2226,
   2108,
   2162,
   2141,
   2202,
   2182,
   2175,
   2146,
   2145,
   2140,
   2180,
   2129,
   2128],
  'incorrect': [448,
   425,
   437,
   534,
   572,
   430,
   384,
   462,
   514,
   494,
   409,
   479,
   411,
   488,
   481,
  

## Paso 8: Evaluación de los resultados

In [18]:
result_original, texts_original = model_original.eval_model(eval_data)
print("📊 Evaluation results:")
print(result_original)

add example index and unique id: 100%|██████████| 8352/8352 [00:00<00:00, 392511.06it/s]


Running Evaluation:   0%|          | 0/5528 [00:00<?, ?it/s]

📊 Evaluation results:
{'correct': 4231, 'similar': 3309, 'incorrect': 812, 'eval_loss': -1.3638708443096508}


In [19]:
result, texts = model.eval_model(eval_data)
print("📊 Evaluation results:")
print(result)

add example index and unique id: 100%|██████████| 8352/8352 [00:00<00:00, 395261.34it/s]


Running Evaluation:   0%|          | 0/5528 [00:00<?, ?it/s]

📊 Evaluation results:
{'correct': 5726, 'similar': 2128, 'incorrect': 498, 'eval_loss': -8.722951621291607}


In [20]:
correct = result_original['correct']
similar = result_original['similar']
incorrect = result_original['incorrect']
total = correct + similar + incorrect

# 1️⃣ Exact Match Accuracy
exact_match = correct / total

# 2️⃣ Weighted Accuracy (partial credit for 'similar')
weighted_accuracy = (correct + 0.5 * similar) / total

# 3️⃣ F1 Score approximation
TP = correct + 0.5 * similar
FN = 0.5 * similar + incorrect
# Assuming FP = 0 (as Simple Transformers counts predictions, not negatives)
precision = TP / TP
recall = TP / (TP + FN)
f1_score = 2 * (precision * recall) / (precision + recall)

# Print results
print(f"Exact Match (EM): {exact_match:.4f} → {exact_match*100:.2f}%")
print(f"Weighted Accuracy: {weighted_accuracy:.4f} → {weighted_accuracy*100:.2f}%")
print(f"F1 Score: {f1_score:.4f} → {f1_score*100:.2f}%")

Exact Match (EM): 0.5066 → 50.66%
Weighted Accuracy: 0.7047 → 70.47%
F1 Score: 0.8268 → 82.68%


In [21]:
correct = result['correct']
similar = result['similar']
incorrect = result['incorrect']
total = correct + similar + incorrect

# 1️⃣ Exact Match Accuracy
exact_match = correct / total

# 2️⃣ Weighted Accuracy (partial credit for 'similar')
weighted_accuracy = (correct + 0.5 * similar) / total

# 3️⃣ F1 Score approximation
TP = correct + 0.5 * similar
FN = 0.5 * similar + incorrect
# Assuming FP = 0 (as Simple Transformers counts predictions, not negatives)
precision = TP / TP
recall = TP / (TP + FN)
f1_score = 2 * (precision * recall) / (precision + recall)

# Print results
print(f"Exact Match (EM): {exact_match:.4f} → {exact_match*100:.2f}%")
print(f"Weighted Accuracy: {weighted_accuracy:.4f} → {weighted_accuracy*100:.2f}%")
print(f"F1 Score: {f1_score:.4f} → {f1_score*100:.2f}%")

Exact Match (EM): 0.6856 → 68.56%
Weighted Accuracy: 0.8130 → 81.30%
F1 Score: 0.8968 → 89.68%


## Paso 9: Guardar los resultados

In [42]:
# Folder to save
local_path = "./QA_model_distill-bert2"
os.makedirs(local_path, exist_ok=True)

# Save the Hugging Face model & tokenizer directly
model.model.save_pretrained(local_path)       # Saves weights + config
model.tokenizer.save_pretrained(local_path)   # Saves vocab + tokenizer config

# Check files
!ls -l ./QA_model

ls: cannot access './QA_model': No such file or directory


In [43]:
shutil.make_archive("QA_model_distill-bert2", 'zip', local_path)
print("✅ Zipped model")
!ls -lh QA_model.zip

✅ Zipped model
ls: cannot access 'QA_model.zip': No such file or directory


In [44]:
files.download("QA_model_distill-bert2.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [66]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [39]:
from huggingface_hub import login

# This will open a prompt for your Hugging Face token
login()

In [40]:
from huggingface_hub import whoami
print(whoami())

{'type': 'user', 'id': '6682cc422b6af3f60a185123', 'name': 'BlueAutomata', 'fullname': 'Guillermo Luigui Ubaldo Nieto Angarita', 'email': 'guillermo.luigui.nieto@gmail.com', 'emailVerified': True, 'canPay': False, 'periodEnd': 1764547199, 'isPro': False, 'avatarUrl': '/avatars/b6cf26e7fac6e034fc5f2b2b87f9ff70.svg', 'orgs': [], 'auth': {'type': 'access_token', 'accessToken': {'displayName': 'write_token', 'role': 'write', 'createdAt': '2025-10-26T19:35:35.613Z'}}}


In [41]:

from huggingface_hub import login, create_repo
from transformers import AutoModelForQuestionAnswering, AutoTokenizer

# 1️⃣ Define your paths and repo name
model_dir = "./outputs"
repo_id = "BlueAutomata/distill-bert-base-spanish-wwm-cased-news-qa-colombia-mexico"

# 2️⃣ Create the repo (won’t fail if it already exists)
create_repo(repo_id, private=False, exist_ok=True)

# 3️⃣ Load your SimpleTransformers model as a standard HF model
hf_model = AutoModelForQuestionAnswering.from_pretrained(model_dir)
hf_tokenizer = AutoTokenizer.from_pretrained(model_dir)

# 4️⃣ Push to the Hugging Face Hub
hf_model.push_to_hub(
    repo_id,
    description="DistilBERT-base Spanish WWM cased model fine-tuned for extractive QA on news articles from Colombia and Mexico.",
    tags=[
        "spanish",
        "question-answering",
        "extractive-qa",
        "distilbert",
        "bert",
        "wwm-cased",
        "colombia",
        "mexico",
        "news-dataset",
        "fine-tuned-model"
    ]
)

hf_tokenizer.push_to_hub(repo_id)

The following layers were not sharded: bert.encoder.layer.*.output.dense.weight, bert.embeddings.token_type_embeddings.weight, bert.encoder.layer.*.attention.self.value.weight, bert.encoder.layer.*.attention.output.LayerNorm.weight, bert.encoder.layer.*.attention.output.LayerNorm.bias, bert.encoder.layer.*.intermediate.dense.weight, bert.encoder.layer.*.output.LayerNorm.weight, bert.encoder.layer.*.attention.output.dense.weight, bert.encoder.layer.*.attention.self.key.bias, qa_outputs.bias, bert.embeddings.position_embeddings.weight, bert.encoder.layer.*.output.dense.bias, bert.encoder.layer.*.output.LayerNorm.bias, bert.encoder.layer.*.attention.output.dense.bias, bert.encoder.layer.*.attention.self.key.weight, bert.embeddings.LayerNorm.weight, bert.encoder.layer.*.attention.self.query.weight, bert.embeddings.LayerNorm.bias, qa_outputs.weight, bert.encoder.layer.*.intermediate.dense.bias, bert.encoder.layer.*.attention.self.query.bias, bert.embeddings.word_embeddings.weight, bert.enco

Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  ...e9cqegd/model.safetensors:  10%|9         | 41.9MB /  437MB            

No files have been modified since last commit. Skipping to prevent empty commit.
No files have been modified since last commit. Skipping to prevent empty commit.


CommitInfo(commit_url='https://huggingface.co/BlueAutomata/distill-bert-base-spanish-wwm-cased-news-qa-colombia-mexico/commit/bd8af2f2ee2cb9253a637639b5daa70360fb1002', commit_message='Upload tokenizer', commit_description='', oid='bd8af2f2ee2cb9253a637639b5daa70360fb1002', pr_url=None, repo_url=RepoUrl('https://huggingface.co/BlueAutomata/distill-bert-base-spanish-wwm-cased-news-qa-colombia-mexico', endpoint='https://huggingface.co', repo_type='model', repo_id='BlueAutomata/distill-bert-base-spanish-wwm-cased-news-qa-colombia-mexico'), pr_revision=None, pr_num=None)

In [46]:

shutil.move("QA_model_distill-bert2.zip", "/content/drive/MyDrive/Thesis_QA_Optimization/Model")

'/content/drive/MyDrive/Thesis_QA_Optimization/Model/QA_model_distill-bert2.zip'

In [47]:
# Path to the folder containing the saved model
model_path = "./QA_model_distill-bert"  # change if different

# Reload the model
my_model = QuestionAnsweringModel(
    "bert",
    model_path,
    use_cuda=True  # set to False if no GPU
)

The following layers were not sharded: bert.encoder.layer.*.output.dense.weight, bert.embeddings.token_type_embeddings.weight, bert.encoder.layer.*.attention.self.value.weight, bert.encoder.layer.*.attention.output.LayerNorm.weight, bert.encoder.layer.*.attention.output.LayerNorm.bias, bert.encoder.layer.*.intermediate.dense.weight, bert.encoder.layer.*.output.LayerNorm.weight, bert.encoder.layer.*.attention.output.dense.weight, bert.encoder.layer.*.attention.self.key.bias, qa_outputs.bias, bert.embeddings.position_embeddings.weight, bert.encoder.layer.*.output.dense.bias, bert.encoder.layer.*.output.LayerNorm.bias, bert.encoder.layer.*.attention.output.dense.bias, bert.encoder.layer.*.attention.self.key.weight, bert.embeddings.LayerNorm.weight, bert.encoder.layer.*.attention.self.query.weight, bert.embeddings.LayerNorm.bias, qa_outputs.weight, bert.encoder.layer.*.intermediate.dense.bias, bert.encoder.layer.*.attention.self.query.bias, bert.embeddings.word_embeddings.weight, bert.enco

In [48]:
# Context & question
context = "Ciudad de México. El capitán de la Secretaría de Marina, Abraham Jeremías Pérez Ramírez, fue hallado muerto en Tamaulipas."
question = "¿Quién fue hallado muerto en Tamaulipas?"

# Prepare input in SimpleTransformers format
to_predict = [
    {
        "context": context,
        "qas": [
            {
                "id": "0",
                "question": question,
                "answers": [{"text": " ", "answer_start": 0}],
                "is_impossible": False
            }
        ]
    }
]

# Run prediction
answers = my_model.predict(to_predict)
print(answers)

add example index and unique id: 100%|██████████| 1/1 [00:00<00:00, 8422.30it/s]


Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

([{'id': '0', 'answer': ['Abraham Jeremías Pérez Ramírez,', 'Ramírez', 'Abraham', 'Abraham Jeremías Pérez', 'Jeremías Pérez Ramírez,', 'Abraham Jeremías', 'Pérez Ramírez', 'El capitán de la Secretaría de Marina, Abraham Jeremías Pérez Ramírez,', 'Marina, Abraham Jeremías Pérez Ramírez,', 'capitán de la Secretaría de Marina, Abraham Jeremías Pérez Ramírez,', 'Abraham Jeremías Pérez Ramírez, fue hallado muerto en Tamaulipas.', 'Ciudad de México. El capitán de la Secretaría de Marina, Abraham Jeremías Pérez Ramírez,', 'México. El capitán de la Secretaría de Marina, Abraham Jeremías Pérez Ramírez,', '', 'Ramírez,', 'Jeremías Pérez', 'Jeremías', 'Pérez', 'El capitán de la Secretaría de Marina, Abraham']}], [{'id': '0', 'probability': [0.9999723004715582, 1.3360977435901276e-05, 7.148124721214904e-06, 2.2779942891231264e-06, 1.8592447521830305e-06, 1.3287423030760395e-06, 9.759334396760402e-07, 2.8290556150848015e-07, 1.7982571593526489e-07, 1.2851564360397842e-07, 6.798804541730626e-08, 5.3

/usr/local/lib/python3.12/dist-packages/simpletransformers/question_answering/question_answering_model.py:1358: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast():


In [34]:
# Load your fine-tuned model from the Hub
qa = pipeline(
    "question-answering",
    model="BlueAutomata/distill-bert-base-spanish-wwm-cased-news-qa-colombia-mexico",
    tokenizer="BlueAutomata/distill-bert-base-spanish-wwm-cased-news-qa-colombia-mexico"
)

# Provide Spanish context
contexto = """
El presidente Gustavo Petro anunció nuevas medidas para impulsar el uso de energías renovables en Colombia,
especialmente en la región del Caribe, donde los proyectos solares y eólicos han ganado protagonismo.
El objetivo del gobierno es reducir las emisiones de carbono en un 30% para el año 2030.
"""

# Ask questions in Spanish
preguntas = [
    "¿Quién anunció nuevas medidas para energías renovables?",
    "¿En qué región se impulsarán los proyectos solares y eólicos?",
    "¿Cuál es el objetivo del gobierno para 2030?"
]

# Evaluate each question
for pregunta in preguntas:
    respuesta = qa(question=pregunta, context=contexto)
    print(f"❓ {pregunta}\n💬 {respuesta['answer']}\n")

config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/437M [00:00<?, ?B/s]

The following layers were not sharded: bert.encoder.layer.*.output.dense.weight, bert.embeddings.token_type_embeddings.weight, bert.encoder.layer.*.attention.self.value.weight, bert.encoder.layer.*.attention.output.LayerNorm.weight, bert.encoder.layer.*.attention.output.LayerNorm.bias, bert.encoder.layer.*.intermediate.dense.weight, bert.encoder.layer.*.output.LayerNorm.weight, bert.encoder.layer.*.attention.output.dense.weight, bert.encoder.layer.*.attention.self.key.bias, qa_outputs.bias, bert.embeddings.position_embeddings.weight, bert.encoder.layer.*.output.dense.bias, bert.encoder.layer.*.output.LayerNorm.bias, bert.encoder.layer.*.attention.output.dense.bias, bert.encoder.layer.*.attention.self.key.weight, bert.embeddings.LayerNorm.weight, bert.encoder.layer.*.attention.self.query.weight, bert.embeddings.LayerNorm.bias, qa_outputs.weight, bert.encoder.layer.*.intermediate.dense.bias, bert.encoder.layer.*.attention.self.query.bias, bert.embeddings.word_embeddings.weight, bert.enco

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

Device set to use cuda:0


❓ ¿Quién anunció nuevas medidas para energías renovables?
💬 Gustavo Petro

❓ ¿En qué región se impulsarán los proyectos solares y eólicos?
💬 Caribe

❓ ¿Cuál es el objetivo del gobierno para 2030?
💬 reducir las emisiones de carbono en un 30%



In [35]:
def flatten_squad(dataset):
    # If the dataset is a dict with "data", extract it
    if isinstance(dataset, dict) and "data" in dataset:
        dataset = dataset["data"]

    new_data = []
    for article in dataset:
        for para in article["paragraphs"]:
            new_data.append({
                "title": article.get("title", ""),
                "context": para["context"],
                "qas": para["qas"]
            })
    return new_data

In [37]:
flat_eval = flatten_squad(eval_dataset)

In [38]:
metric = evaluate.load("squad")

In [50]:
from transformers import pipeline
import evaluate

qa_pipeline = pipeline(
    "question-answering",
    model="BlueAutomata/distill-bert-base-spanish-wwm-cased-news-qa-colombia-mexico",
    tokenizer="BlueAutomata/distill-bert-base-spanish-wwm-cased-news-qa-colombia-mexico"
)



The following layers were not sharded: bert.encoder.layer.*.output.dense.weight, bert.embeddings.token_type_embeddings.weight, bert.encoder.layer.*.attention.self.value.weight, bert.encoder.layer.*.attention.output.LayerNorm.weight, bert.encoder.layer.*.attention.output.LayerNorm.bias, bert.encoder.layer.*.intermediate.dense.weight, bert.encoder.layer.*.output.LayerNorm.weight, bert.encoder.layer.*.attention.output.dense.weight, bert.encoder.layer.*.attention.self.key.bias, qa_outputs.bias, bert.embeddings.position_embeddings.weight, bert.encoder.layer.*.output.dense.bias, bert.encoder.layer.*.output.LayerNorm.bias, bert.encoder.layer.*.attention.output.dense.bias, bert.encoder.layer.*.attention.self.key.weight, bert.embeddings.LayerNorm.weight, bert.encoder.layer.*.attention.self.query.weight, bert.embeddings.LayerNorm.bias, qa_outputs.weight, bert.encoder.layer.*.intermediate.dense.bias, bert.encoder.layer.*.attention.self.query.bias, bert.embeddings.word_embeddings.weight, bert.enco

In [49]:
from transformers import pipeline
import evaluate

qa_pipeline_original = pipeline(
    "question-answering",
    model="mrm8488/distill-bert-base-spanish-wwm-cased-finetuned-spa-squad2-es",
    tokenizer="mrm8488/distill-bert-base-spanish-wwm-cased-finetuned-spa-squad2-es"
)


The following layers were not sharded: bert.encoder.layer.*.output.dense.weight, bert.embeddings.token_type_embeddings.weight, bert.encoder.layer.*.attention.self.value.weight, bert.encoder.layer.*.attention.output.LayerNorm.weight, bert.encoder.layer.*.attention.output.LayerNorm.bias, bert.encoder.layer.*.intermediate.dense.weight, bert.encoder.layer.*.output.LayerNorm.weight, bert.encoder.layer.*.attention.output.dense.weight, bert.encoder.layer.*.attention.self.key.bias, qa_outputs.bias, bert.embeddings.position_embeddings.weight, bert.encoder.layer.*.output.dense.bias, bert.encoder.layer.*.output.LayerNorm.bias, bert.encoder.layer.*.attention.output.dense.bias, bert.encoder.layer.*.attention.self.key.weight, bert.embeddings.LayerNorm.weight, bert.encoder.layer.*.attention.self.query.weight, bert.embeddings.LayerNorm.bias, qa_outputs.weight, bert.encoder.layer.*.intermediate.dense.bias, bert.encoder.layer.*.attention.self.query.bias, bert.embeddings.word_embeddings.weight, bert.enco

In [51]:
predictions_original = []
references_original = []

for ex in flat_eval:
    context = ex["context"]
    for qa in ex["qas"]:
        if qa["is_impossible"]:
            continue  # Skip unanswerable questions

        # Run QA prediction
        pred = qa_pipeline_original(question=qa["question"], context=context)

        # Collect prediction and reference
        predictions_original.append({
            "id": qa["id"],
            "prediction_text": pred["answer"]
        })

        references_original.append({
            "id": qa["id"],
            "answers": {
                "text": [a["text"] for a in qa["answers"]],
                "answer_start": [a["answer_start"] for a in qa["answers"]]
            }
        })

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


In [52]:
predictions = []
references = []

for ex in flat_eval:
    context = ex["context"]
    for qa in ex["qas"]:
        if qa["is_impossible"]:
            continue  # Skip unanswerable questions

        # Run QA prediction
        pred = qa_pipeline(question=qa["question"], context=context)

        # Collect prediction and reference
        predictions.append({
            "id": qa["id"],
            "prediction_text": pred["answer"]
        })

        references.append({
            "id": qa["id"],
            "answers": {
                "text": [a["text"] for a in qa["answers"]],
                "answer_start": [a["answer_start"] for a in qa["answers"]]
            }
        })


In [53]:
results = metric.compute(predictions=predictions_original, references=references_original)
print("📊 Evaluation results:")
print(f"Exact Match: {results['exact_match']:.2f}")
print(f"F1 Score: {results['f1']:.2f}")

📊 Evaluation results:
Exact Match: 59.24
F1 Score: 74.51


In [54]:
results = metric.compute(predictions=predictions, references=references)
print("📊 Evaluation results:")
print(f"Exact Match: {results['exact_match']:.2f}")
print(f"F1 Score: {results['f1']:.2f}")

📊 Evaluation results:
Exact Match: 75.10
F1 Score: 83.60


In [55]:
import evaluate
metric = evaluate.load("squad_v2")

In [ ]:
from tqdm import tqdm

predictions_original = []
references_original = []

for ex in tqdm(flat_eval):
    for qa in ex["qas"]:
        if qa["is_impossible"]:
            # Questions that have no valid answer in the text
            predictions_original.append({
                "id": qa["id"],
                "prediction_text": "",
                "no_answer_probability": 1.0   # fully confident it's unanswerable
            })
            references_original.append({
                "id": qa["id"],
                "answers": {"text": [], "answer_start": []}
            })
        else:
            # Normal (answerable) questions
            pred = qa_pipeline_original(question=qa["question"], context=ex["context"])

            predictions_original.append({
                "id": qa["id"],
                "prediction_text": pred["answer"],
                # Use model confidence inversely as no-answer probability
                "no_answer_probability": 1.0 - pred.get("score", 0.0)
            })

            references_original.append({
                "id": qa["id"],
                "answers": {
                    "text": [a["text"] for a in qa["answers"]],
                    "answer_start": [a["answer_start"] for a in qa["answers"]]
                }
            })

In [56]:
from tqdm import tqdm

predictions = []
references = []

for ex in tqdm(flat_eval):
    for qa in ex["qas"]:
        if qa["is_impossible"]:
            # Questions that have no valid answer in the text
            predictions.append({
                "id": qa["id"],
                "prediction_text": "",
                "no_answer_probability": 1.0   # fully confident it's unanswerable
            })
            references.append({
                "id": qa["id"],
                "answers": {"text": [], "answer_start": []}
            })
        else:
            # Normal (answerable) questions
            pred = qa_pipeline(question=qa["question"], context=ex["context"])

            predictions.append({
                "id": qa["id"],
                "prediction_text": pred["answer"],
                # Use model confidence inversely as no-answer probability
                "no_answer_probability": 1.0 - pred.get("score", 0.0)
            })

            references.append({
                "id": qa["id"],
                "answers": {
                    "text": [a["text"] for a in qa["answers"]],
                    "answer_start": [a["answer_start"] for a in qa["answers"]]
                }
            })

100%|██████████| 3846/3846 [07:59<00:00,  8.01it/s]


In [57]:
predictions_original

[{'id': 'qa-20530', 'prediction_text': 'Blue Velvet Revisited'},
 {'id': 'qa-20528', 'prediction_text': 'Tuxedomoon'},
 {'id': 'qa-27430', 'prediction_text': '(National Poison Database System'},
 {'id': 'qa-27428', 'prediction_text': 'difenhidramina'},
 {'id': 'qa-21633', 'prediction_text': 'Tren Maya'},
 {'id': 'qa-20670', 'prediction_text': 'Citlali'},
 {'id': 'qa-21307', 'prediction_text': 'Argentina'},
 {'id': 'qa-21250', 'prediction_text': 'adicciones'},
 {'id': 'qa-22771', 'prediction_text': 'presidenta Claudia Sheinbaum Pardo'},
 {'id': 'qa-22775', 'prediction_text': '(15 mil 645 millones de dólares)'},
 {'id': 'qa-6164',
  'prediction_text': 'secretaria de Seguridad Nacional de Estados Unidos'},
 {'id': 'qa-25169', 'prediction_text': 'Guerreros Buscadores'},
 {'id': 'qa-25882',
  'prediction_text': 'una nueva sentencia con agravantes contra el feminicidio'},
 {'id': 'qa-23223', 'prediction_text': 'líderes árabes'},
 {'id': 'qa-25089',
  'prediction_text': 'presidente Andrés Man

In [59]:
results = metric.compute(predictions=predictions_original, references=references_original)

print("📊 Evaluation results:")
print(f"Exact Match: {results['exact']:.2f}")
print(f"F1 Score: {results['f1']:.2f}")

KeyError: 'no_answer_probability'

In [60]:
results = metric.compute(predictions=predictions, references=references)
print("📊 Evaluation results:")
print(f"Exact Match: {results['exact']:.2f}")
print(f"F1 Score: {results['f1']:.2f}")

📊 Evaluation results:
Exact Match: 82.79
F1 Score: 88.67


In [61]:
!mv outputs outputs_distill-bert2
!zip -r outputs_distill-bert2.zip outputs_distill-bert2

  adding: outputs_distill-bert2/ (stored 0%)
  adding: outputs_distill-bert2/checkpoint-10500/ (stored 0%)
  adding: outputs_distill-bert2/checkpoint-10500/config.json (deflated 49%)
  adding: outputs_distill-bert2/checkpoint-10500/optimizer.pt (deflated 10%)
  adding: outputs_distill-bert2/checkpoint-10500/model.safetensors (deflated 7%)
  adding: outputs_distill-bert2/checkpoint-10500/training_args.bin (deflated 53%)
  adding: outputs_distill-bert2/checkpoint-10500/special_tokens_map.json (deflated 42%)
  adding: outputs_distill-bert2/checkpoint-10500/vocab.txt (deflated 54%)
  adding: outputs_distill-bert2/checkpoint-10500/model_args.json (deflated 60%)
  adding: outputs_distill-bert2/checkpoint-10500/scheduler.pt (deflated 61%)
  adding: outputs_distill-bert2/checkpoint-10500/tokenizer_config.json (deflated 74%)
  adding: outputs_distill-bert2/checkpoint-10500/eval_results.txt (deflated 10%)
  adding: outputs_distill-bert2/checkpoint-12000/ (stored 0%)
  adding: outputs_distill-ber

In [63]:
shutil.move("outputs_distill-bert2.zip", "/content/drive/MyDrive/Thesis_QA_Optimization/Model")

'/content/drive/MyDrive/Thesis_QA_Optimization/Model/outputs_distill-bert2.zip'